<a href="https://colab.research.google.com/github/FIJY/hh/blob/master/colab/hh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Парсинг с сайта hh <br>
Ссылка на видео парсинга hh https://yandex.ru/video/preview/11468178118806870731

In [1]:
# Устанавливаем библиотеки
# requests для отправки запросов
# BeautifulSoup для разбора полученных данных
# lxml для парсинга
# fake_useragent для создания заголовков
!pip install requests
!pip install bs4
!pip install lxml
!pip install fake_useragent

In [ ]:
# Импортируем библиотеки
import requests
from bs4 import BeautifulSoup
import fake_useragent
import time
import json
import pandas as pd
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
folder_path = '/content/drive/My Drive/Папка_проекта/'


In [ ]:
# Общий User-Agent для всех запросов
ua = fake_useragent.UserAgent()

def get_links(text, experience=None, age_from=None, age_to=None, salary=None, education=None):
    # Строим URL
    url = f"https://hh.ru/search/resume?text={text}&area=1&isDefaultArea=true&exp_period=all_time&logic=normal&pos=full_text"
    # Добавляем параметры
    if experience:
        url += f"&experience={experience}"
    if age_from:
        url += f"&age_from={age_from}"
    if age_to:
        url += f"&age_to={age_to}"
    if salary:
        url += f"&salary={salary}"
    if education:
        url += f"&education={education}"

    data = requests.get(url, headers={"user-agent": ua.random})
    if data.status_code != 200:
        print(f"Ошибка запроса: HTTP {data.status_code}")
        return []

    soup = BeautifulSoup(data.content, "lxml")

    # Определение количества страниц
    try:
        page_count = int(soup.find("div", attrs={"class":"pager"}).find_all("span", recursive=False)[-1].find("a").find("span").text)
    except Exception as e:
        print("Ошибка при получении количества страниц: ", e)
        return []

    links = []
    for page in range(1, page_count + 1):
        try:
            data = requests.get(
                url=f"{url}&page={page}",
                headers={"user-agent": ua.random}
            )
            if data.status_code != 200:
                print(f"Пропуск страницы {page} из-за ошибки HTTP {data.status_code}")
                continue
            soup = BeautifulSoup(data.content, "lxml")
            for a in soup.find_all("a", attrs={"data-qa":"serp-item__title"}):
                links.append(f"https://hh.ru{a.attrs['href'].split('?')[0]}")
        except Exception as e:
            print(f"Ошибка при обработке страницы {page}: {e}")
        time.sleep(1)
    return links


In [ ]:
# Уточнение запроса
# Давайте рассмотрим, как можно уточнить запрос, используя различные параметры:

# Текст запроса (text): Основной поисковый запрос, например, "Python разработчик".
# Опыт работы (experience): Значения могут быть, например, noExperience, between1And3, between3And6, moreThan6.
# Возраст от и до (age_from, age_to): Числовые значения, указывающие возрастные рамки.
# Зарплата от (salary): Минимальная зарплатная ожидаемость, например, from50000.
# Образование (education): Может принимать значения как secondary, specialSecondary, higher.


link_resume = [a for a in get_links("Python разработчик", salary="from50000")]


Пропуск страницы 250 из-за ошибки HTTP 404


In [ ]:
len (link_resume)

4978

In [ ]:
# Запишем полученные ссылки в файл
with open ('link_resume.txt', 'w') as file:
  for resume in link_resume:
    file.write(f"{resume} ")

In [ ]:
# Переходим по любой ссылке в резюме, с указанной зарплатой
# Смотрим код и копируем class заголовка (для этого нажмем "проверить" и выделим заголовок, т.е. название позиции, например Python developer)
# Аналогично скопируем class зарплаты
# Также ищем class ключевых навыков

In [ ]:
def get_resume(link):
  ua = fake_useragent.UserAgent() # Создадим юзер-агента
  # Делаем запрос к переданной ссылке
  data = requests.get(
      url = link,
      headers={"user-agent":ua.random}
  )
  if data.status_code != 200:
    return

  soup = BeautifulSoup(data.content, "lxml")

  # Спарсим данные о соискателе
  reperson = re.compile(r'resume-personal.*') # Регулярное выражение для поиска всех атрибутов, содержащих "resume-personal"
  try:
    persons = [person.text.replace("\xa0", " ") for person in soup.find_all(attrs={"data-qa":reperson})]
  except:
    persons = ""

  # Спарсим готовность к переезду, командировкам
  try:
    moving = soup.find(attrs={"class":"bloko-translate-guard"}).text.replace("\xa0", " ")
  except:
    moving = ""

  # Спарсим название позиции
  try:
    name = soup.find(attrs={"class":"resume-block__title-text"}).text
  except:
    name = ""

  # Спарсим зарплату и уберем лишние символы (\u2009, \xa0)
  try:
    salary = soup.find(attrs={"class":"resume-block__salary"}).text.replace("\u2009","").replace("\xa0", " ")
  except:
    salary = ""

  # Спарсим специализации и график работы
  try:
    specialization_schedule = re.sub(r'([А-ЯA-Z])', r' \1', soup.find(attrs={"class":"resume-block-container"}).text).strip()
  except:
    specialization_schedule = ""

  # Спарсим опыт работы
  try:
    experience = soup.find(attrs={"data-qa":"resume-block-experience"}).text.replace("\xa0", " ")
  except:
    experience = ""

  # Находим атрибут, где находятся все ключевые навыки
  try:
    tags = [tag.text for tag in soup.find(attrs={"data-qa":"skills-table"}).find_all(attrs={"class":"bloko-tag__section_text"})]
  except:
    tags = []

  # Добавляем линк в словарь
  resume = {
      "persons": persons,
      "moving": moving,
      "name": name,
      "salary": salary,
      "specialization_schedule": specialization_schedule,
      "experience": experience,
      "tags": tags,
      "link": link
  }
  return resume


In [ ]:
# Здесь в зарплате видны лишние символы (\u2009, \xa0), скопируем их и вставим в созданную выше функцию, чтобы их убрать
for a in link_resume[:10]:
  print(get_resume(a))
  time.sleep(1)

{'persons': ['Мужчина', '24 года', '10 декабря 1999', 'Новосибирск', 'м. Заельцовская'], 'moving': 'Новосибирск, м. Заельцовская, готов к переезду (Москва, Санкт-Петербург, Другие регионы), не готов к командировкам', 'name': 'Python developer', 'salary': '', 'specialization_schedule': 'Специализации: Программист, разработчик Занятость: полная занятость График работы: полный день, сменный график, гибкий график, удаленная работа', 'experience': 'Опыт работы 2 года 4 месяца Март 2023 — по настоящее время1 год 2 месяцаWhitelist CapitalЕкатеринбург, whitelist.capital/Информационные технологии, системная интеграция, интернет... Показать ещеPython developerBackend developerАвгуст 2022 — Октябрь 20223 месяцаВЕБ РИЭЛТИМосква, idaproject.com/Информационные технологии, системная интеграция, интернет... Показать ещеПрограммист Python1. Backend\nСтек технологий:\nPython, Django + DRF, Docker, PostgreSQL, Redis, Celery, unit test.\n\nФункционал:\n- Разработка backend части сайта, обсуждение с fronte

In [ ]:
# Вот, что получилось, когда убрали лишние символы в зарплате
for a in link_resume[:10]:
  print(get_resume(a))
  time.sleep(1)

{'persons': ['Мужчина', '24 года', '10 декабря 1999', 'Новосибирск', 'м. Заельцовская'], 'moving': 'Новосибирск, м. Заельцовская, готов к переезду (Москва, Санкт-Петербург, Другие регионы), не готов к командировкам', 'name': 'Python developer', 'salary': '', 'specialization_schedule': 'Специализации: Программист, разработчик Занятость: полная занятость График работы: полный день, сменный график, гибкий график, удаленная работа', 'experience': 'Опыт работы 2 года 4 месяца Март 2023 — по настоящее время1 год 2 месяцаWhitelist CapitalЕкатеринбург, whitelist.capital/Информационные технологии, системная интеграция, интернет... Показать ещеPython developerBackend developerАвгуст 2022 — Октябрь 20223 месяцаВЕБ РИЭЛТИМосква, idaproject.com/Информационные технологии, системная интеграция, интернет... Показать ещеПрограммист Python1. Backend\nСтек технологий:\nPython, Django + DRF, Docker, PostgreSQL, Redis, Celery, unit test.\n\nФункционал:\n- Разработка backend части сайта, обсуждение с fronte

In [ ]:
# Добавили навыки
for a in link_resume[:10]:
  print(get_resume(a))
  time.sleep(1)

{'persons': [], 'moving': '', 'name': '', 'salary': '', 'specialization_schedule': '', 'experience': '', 'tags': [], 'link': 'https://hh.ru/resume/7b1a872100086178440039ed1f61766c623747'}
{'persons': ['Мужчина', '32 года', '6 января 1992', 'Москва', 'м. Тульская'], 'moving': 'Москва, м. Тульская, готов к переезду, готов к командировкам', 'name': 'Программист Python', 'salary': '350000 ₽ на руки', 'specialization_schedule': '350\u2009000\xa0₽ на\xa0руки', 'experience': 'Опыт работы 8 лет 9 месяцев Ноябрь 2016 — по настоящее время7 лет 6 месяцевЯндексЕкатеринбург, www.yandex.ruИнформационные технологии, системная интеграция, интернет... Показать ещеПрограммист-разработчикЯндекс.Авиабилеты - разработка бэкенда\n- Python, Django, Celery\n- GoLang\n- Yandex Database (YDB)\n- YT (внутренняя MapReduce-система)\n- Logbroker (внутренний аналог Kafka)\n- Mysql, Postgresql, Mongo DB, Memcached, Redis, RabbitMQ, SentryИюль 2016 — Сентябрь 20163 месяцаЯндексЕкатеринбург, www.yandex.ruИнформационные

In [ ]:
for a in link_resume[:10]:
  print(get_resume(a))
  time.sleep(1)

{'persons': ['Мужчина', '24 года', '10 декабря 1999', 'Новосибирск', 'м. Заельцовская'], 'moving': 'Новосибирск, м. Заельцовская, готов к переезду (Москва, Санкт-Петербург, Другие регионы), не готов к командировкам', 'name': 'Python developer', 'salary': '', 'specialization_schedule': 'Специализации: Программист, разработчик Занятость: полная занятость График работы: полный день, сменный график, гибкий график, удаленная работа', 'experience': 'Опыт работы 2 года 4 месяца Март 2023 — по настоящее время1 год 2 месяцаWhitelist CapitalЕкатеринбург, whitelist.capital/Информационные технологии, системная интеграция, интернет... Показать ещеPython developerBackend developerАвгуст 2022 — Октябрь 20223 месяцаВЕБ РИЭЛТИМосква, idaproject.com/Информационные технологии, системная интеграция, интернет... Показать ещеПрограммист Python1. Backend\nСтек технологий:\nPython, Django + DRF, Docker, PostgreSQL, Redis, Celery, unit test.\n\nФункционал:\n- Разработка backend части сайта, обсуждение с fronte

In [ ]:
# Сохраняем результат в файл .csv

data = []
for link in link_resume[:10]:
    resume_data = get_resume(link)
    if resume_data:  # Убедитесь, что данные были получены
        data.append(resume_data)
    time.sleep(1)  # Задержка для снижения нагрузки на сервер

# Преобразуем список словарей в DataFrame
df = pd.DataFrame(data)

# Сохраняем DataFrame в CSV файл
df.to_csv("data.csv", index=False, encoding='utf-8-sig')  # index=False означает, что индексы не будут сохранены в файл


In [ ]:
import pandas as pd

# Предположим, df - ваш DataFrame с результатами
# Сохранение данных в CSV файл
output_file = folder_path + 'output_data.csv'
df.to_csv(output_file, index=False)


In [ ]:
# Извлекаем данные из файла .csv с помощью pandas
data_df = pd.read_csv("data.csv")
data_df.head()

,persons,moving,name,salary,specialization_schedule,experience,tags,link
0,"['Мужчина', '24 года', '10 декабря 1999', 'Нов...","Новосибирск, м. Заельцовская, готов к переезду...",Python developer,NaN,"Специализации: Программист, разработчик Занято...",Опыт работы 2 года 4 месяца Март 2023 — по нас...,"['Git', 'Python', 'ООП', 'Django Framework', '...",https://hh.ru/resume/7b1a872100086178440039ed1...
1,"['Мужчина', '32 года', '6 января 1992', 'Москв...","Москва, м. Тульская, готов к переезду, готов к...",Программист Python,350000 ₽ на руки,350 000 ₽ на руки,Опыт работы 8 лет 9 месяцев Ноябрь 2016 — по н...,"['Python', 'Machine Learning', 'Linux', 'Compu...",https://hh.ru/resume/fb7ea8c800035e24280039ed1...
2,"['Мужчина', '23 года', '23 июня 2000', 'Москва...","Москва, м. Сходненская, не готов к переезду, г...",Python-разработчик/ Python developer,NaN,"Специализации: Программист, разработчик Занято...",Опыт работы 4 года 2 месяца Декабрь 2020 — по ...,"['Исполнительность', 'Поиск информации в интер...",https://hh.ru/resume/6a9abec70008de01cc0039ed1...
3,"['Мужчина', '24 года', '17 февраля 2000', 'Мос...","Москва, не готов к переезду, не готов к команд...",Программист,NaN,"Специализации: Программист, разработчик Занято...",Опыт работы 3 года 4 месяца Июль 2023 — по нас...,"[' Python', 'JavaScript', 'Git', 'Bash', 'Linu...",https://hh.ru/resume/bee0660c00080a04f10039ed1...
4,"['Мужчина', '31 год', '28 февраля 1993', 'Влад...","Владивосток, готов к переезду (Беларусь, Москв...",Программист Python,NaN,"Специализации: Программист, разработчик Занято...",Опыт работы 7 лет 8 месяцев Сентябрь 2016 — по...,[],https://hh.ru/resume/b20787e00003b6474f0039ed1...
